In [109]:
import pandas as pd
import numpy as np

# Charger les fichiers CSV
weather_df = pd.read_csv('cleaned_weather 1.csv')
planes_df = pd.read_csv('cleaned_planes 1.csv')
flights = pd.read_csv('cleaned_flights 1.csv')
airports_df = pd.read_csv('cleaned_airports 1.csv')
airlines_df = pd.read_csv('cleaned_airlines 1.csv')



In [54]:
flights.columns

Index(['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour'],
      dtype='object')

In [81]:
flights_df = flights.dropna(subset=['dep_delay'])


In [110]:
flights_weather_df = pd.merge(flights_df, weather_df, on=['time_hour', 'origin'], how='left')
flights_weather_airlines_df = pd.merge(flights_weather_df, airlines_df, on='carrier', how='left')
flights_weather_airlines_planes_df = pd.merge(flights_weather_airlines_df, planes_df, on='tailnum', how='left')
final_df = pd.merge(flights_weather_airlines_planes_df, airports_df, left_on='origin', right_on='FAA', how='left', suffixes=('', '_origin'))
final_df = pd.merge(final_df, airports_df, left_on='dest', right_on='FAA', how='left', suffixes=('', '_dest'))
final_df = final_df.drop(columns=['sched_dep_time', 'sched_arr_time', 'tailnum', 'FAA', 'FAA_dest'])


In [111]:
final_df['temp'].fillna(weather_df['temp'].mean(), inplace=True)
final_df['wind_speed'].fillna(weather_df['wind_speed'].mean(), inplace=True)
final_df['dewp'].fillna(weather_df['dewp'].mean(), inplace=True)
final_df['humid'].fillna(weather_df['humid'].mean(), inplace=True)
final_df['wind_dir'].fillna(weather_df['wind_dir'].mean(), inplace=True)
final_df['precip'].fillna(weather_df['precip'].mean(), inplace=True)
final_df['pressure'].fillna(weather_df['pressure'].mean(), inplace=True)
final_df['visib'].fillna(weather_df['visib'].mean(), inplace=True)
final_df['dep_delay'].fillna(0, inplace=True)
#df_flights_weather_planes_2 = flights_weather_planes.dropna(subset=['dep_delay'])

In [112]:
features = ['year_x', 'month_x', 'day_x', 'hour_x', 'minute', 'arr_delay', 'air_time', 'distance', 'carrier', 'origin', 'dest', 'temp', 'dewp', 'humid', 'wind_dir', 'wind_speed', 'precip', 'pressure', 'visib']
target = 'dep_delay'

X = final_df[features]
y = final_df[target]


In [57]:
X_pr = flights[['carrier', 'origin', 'dest']]
y_pr = flights['dep_delay']

In [63]:
X_pri = X_pr[:-2997]

In [61]:
y_pri = y_pr.dropna()

In [85]:
X_prime = X.dropna()

In [92]:
y_prime = y[:-(587)]

In [93]:
y_prime

0           2.0
1           4.0
2           2.0
3          -6.0
4          -4.0
          ...  
185682      4.0
185683      6.0
185684    167.0
185685     11.0
185686     22.0
Name: dep_delay, Length: 185687, dtype: float64

In [94]:
X_prime

,year_x,month_x,day_x,hour_x,minute,arr_delay,air_time,distance,carrier,origin,dest,temp,dewp,humid,wind_dir,wind_speed,precip,pressure,visib
0,2021,1,1,5,15,11.0,227.0,1400,UA,EWR,IAH,55.203515,41.385399,62.347322,198.066661,10.395868,0.002726,1017.895175,9.204828
1,2021,1,1,5,29,20.0,227.0,1416,UA,LGA,IAH,55.203515,41.385399,62.347322,198.066661,10.395868,0.002726,1017.895175,9.204828
2,2021,1,1,5,40,33.0,160.0,1089,AA,JFK,MIA,55.203515,41.385399,62.347322,198.066661,10.395868,0.002726,1017.895175,9.204828
3,2021,1,1,6,0,-25.0,116.0,762,DL,LGA,ATL,39.920000,26.060000,57.330000,260.000000,13.809360,0.000000,1011.900000,10.000000
4,2021,1,1,5,58,12.0,150.0,719,UA,EWR,ORD,55.203515,41.385399,62.347322,198.066661,10.395868,0.002726,1017.895175,9.204828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186269,2021,7,3,11,0,-9.0,135.0,944,DL,JFK,MCO,75.020000,69.980000,84.340000,200.000000,6.904680,0.000000,1024.700000,10.000000
186270,2021,7,3,11,0,-6.0,43.0,214,US,LGA,DCA,78.080000,69.080000,73.880000,190.000000,6.904680,0.000000,1023.900000,10.000000
186271,2021,7,3,11,10,25.0,61.0,288,9E,JFK,RIC,75.020000,69.980000,84.340000,200.000000,6.904680,0.000000,1024.700000,10.000000
186272,2021,7,3,10,55,37.0,176.0,1076,B6,LGA,FLL,75.920000,69.080000,79.370000,210.000000,8.055460,0.000000,1023.600000,10.000000


In [113]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [114]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_features = [ 'temp', 'dewp', 'humid', 'wind_dir', 'wind_speed', 'precip', 'pressure', 'visib']
categorical_features = ['carrier', 'origin', 'dest']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)


In [115]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

pipeline_lr = Pipeline(steps=[('preprocessor', preprocessor), ('model', LinearRegression())])
#pipeline_dt = Pipeline(steps=[('preprocessor', preprocessor), ('model', DecisionTreeRegressor(random_state=42))])
#pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestRegressor(random_state=42))])

pipeline_lr.fit(X_train, y_train)
#pipeline_dt.fit(X_train, y_train)
#pipeline_rf.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['temp', 'dewp', 'humid',
                                                   'wind_dir', 'wind_speed',
                                                   'precip', 'pressure',
                                                   'visib']),
                                                 ('cat', OneHotEncoder(),
                                                  ['carrier', 'origin',
                                                   'dest'])])),
                ('model', LinearRegression())])

In [116]:
# Evaluate the models
y_pred_lr = pipeline_lr.predict(X_test)

In [117]:
from sklearn.metrics import mean_squared_error,r2_score
rmse_lr = mean_squared_error(y_test, y_pred_lr, squared=False)
rmse_lr

38.83854387671161

In [118]:
r_squared = r2_score(y_test, y_pred_lr)
r_squared

0.04940993141709071

In [71]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor

# Créer le pipeline pour le modèle de régression par gradient boosting
pipeline_gb = Pipeline(steps=[('preprocessor', preprocessor), ('model', GradientBoostingRegressor(random_state=42))])

# Entraîner le modèle
pipeline_gb.fit(X_train, y_train)

# Évaluer le modèle
y_pred_gb = pipeline_gb.predict(X_test)
rmse_gb = mean_squared_error(y_test, y_pred_gb, squared=False)

print(f'RMSE de Gradient Boosting: {rmse_gb}')

RMSE de Gradient Boosting: 39.844951209452


In [ ]:
#pipeline_dt = Pipeline(steps=[('preprocessor', preprocessor), ('model', DecisionTreeRegressor(random_state=42))])
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestRegressor(random_state=42))])

#pipeline_lr.fit(X_train, y_train)
#pipeline_dt.fit(X_train, y_train)
pipeline_rf.fit(X_train, y_train)


In [73]:
# Evaluate the models
y_pred_rf = pipeline_rf.predict(X_test)
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
rmse_rf

39.87290865237647

In [49]:
from sklearn.model_selection import  GridSearchCV
# Créer le pipeline pour le modèle de régression par gradient boosting
pipeline_gb = Pipeline(steps=[('preprocessor', preprocessor), ('model', GradientBoostingRegressor(random_state=42))])

# Définir les paramètres pour GridSearchCV
param_grid = {
    'model__n_estimators': [100, 200],
    'model__learning_rate': [0.01, 0.1],
    'model__max_depth': [3, 5],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2],
    'model__subsample': [0.8, 1.0]
}

# Appliquer GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(pipeline_gb, param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print(f'Best parameters found: {grid_search.best_params_}')

# Entraîner le modèle avec les meilleurs paramètres
best_pipeline_gb = grid_search.best_estimator_
best_pipeline_gb.fit(X_train, y_train)

# Évaluer le modèle
y_pred_gb = best_pipeline_gb.predict(X_test)
rmse_gb = mean_squared_error(y_test, y_pred_gb, squared=False)

print(f'RMSE de Gradient Boosting après optimisation: {rmse_gb}')

KeyboardInterrupt: 